In [1]:
from azure.mgmt.resource import SubscriptionClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

In [2]:
# Subscription ID
subscription_client = SubscriptionClient(credential)

# Liste aller Abonnements abrufen, auf die der authentifizierte Account Zugriff hat
subscriptions = subscription_client.subscriptions.list()

# Die erste Subscription ID speichern
subscription_id = next(subscriptions).subscription_id

print(f"Subscription ID: {subscription_id}")

Subscription ID: 5b7d2bfa-802a-431d-8c31-1c4069bbb309


In [3]:
from azure.mgmt.resource import ResourceManagementClient

resource_client = ResourceManagementClient(credential, subscription_id)

# List all resource groups
resource_groups = resource_client.resource_groups.list()

# Get the name of the first resource group
resource_group_name = next(resource_groups).name

print(f"Using existing resource group: {resource_group_name}")

Using existing resource group: aind-257943


In [4]:
from azure.mgmt.storage import StorageManagementClient
from azure.mgmt.storage.models import StorageAccountCreateParameters, Sku

storage_client = StorageManagementClient(credential, subscription_id)

storage_account_name = "boardingkioskstorageacc"
location = "eastus"

In [5]:
# Check if the storage account name is available
availability_result = storage_client.storage_accounts.check_name_availability(
    {"name": storage_account_name}
)

if not availability_result.name_available:
    print(f"Storage account name '{storage_account_name}' is not available.")
    exit(1)

Storage account name 'boardingkioskstorageacc' is not available.


In [14]:
from azure.mgmt.storage import StorageManagementClient
from azure.mgmt.storage.models import StorageAccountCreateParameters, Sku

# Create a StorageManagementClient
storage_client = StorageManagementClient(credential, subscription_id)

# Check if the storage account already exists
storage_account_list = storage_client.storage_accounts.list_by_resource_group(
    resource_group_name
)
storage_account_exists = any(
    account.name == storage_account_name for account in storage_account_list
)

if not storage_account_exists:
    # Create the storage account
    storage_account = storage_client.storage_accounts.begin_create(
        resource_group_name,
        storage_account_name,
        StorageAccountCreateParameters(
            sku=Sku(name="Standard_LRS"), kind="StorageV2", location=location
        ),
    ).result()

    print(
        f"Storage account '{storage_account.name}' created in location '{storage_account.location}'."
    )


else:
    print(f"Storage account '{storage_account_name}' already exists.")

Storage account 'boardingkioskstorageacc' already exists.
